In [1]:
%load_ext autoreload
%autoreload 2
# pretrain MLP using the new dataloader

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["KERAS_BACKEND"] = "jax"

from typing import List
import h5py
from torch.utils.data import Dataset, DataLoader
import numpy as np
import preprocessing
from tqdm import tqdm
import keras_core as keras
from keras_core import layers
from keras_core import ops
import matplotlib.pyplot as plt

from data_utils import preprocess_data, H5Dataset

Using JAX backend.


2023-10-24 01:22:03.095881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_dir = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train/"
train_dir_preprocess = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/"
train_filepaths = [train_dir + x for x in os.listdir(train_dir)]
# preprocess_data(train_filepaths, train_dir_preprocess, force=False)
train_preprocess_file_names = os.listdir(train_dir_preprocess)
train_preprocess_filepaths = [train_dir_preprocess + name for name in train_preprocess_file_names]

In [3]:
train_dataset = H5Dataset(train_preprocess_filepaths[:1], transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=False)
print("Num train samples:", len(train_dataset))

Num train samples: 5000000


In [4]:
test_dataset = H5Dataset(train_preprocess_filepaths[5:6], transform=None)
test_dataloader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
print("Num test samples", len(test_dataset))

Num test samples 5000000


In [5]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import Flatten

In [6]:
# model = Sequential()
# model.add(Dense(64, input_shape=(200*7,), activation='relu'))
# model.add(Flatten())
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "AUC"])

In [7]:
# history = model.fit(train_dataloader, batch_size=256, validation_data=test_dataloader, epochs=1)

In [8]:
model = keras.Sequential([
  keras.Input(shape=(200 * 7,)),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(8, activation='relu'),
  layers.Dense(1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"])

2023-10-19 19:33:54.779136: W external/xla/xla/service/gpu/nvptx_compiler.cc:596] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.3.52). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [9]:
history = model.fit(train_dataloader, batch_size=1024, validation_data=test_dataloader, epochs=1)
model.evaluate(test_dataloader)

In [ ]:
history